In [1]:
import codecs
import re

import numpy as np
import pandas as pd
import spacy
from tqdm import tqdm

In [2]:
nlp = spacy.load('en')

In [22]:
docs_index = pd.read_csv('../../data/documents.csv')
questions = pd.read_csv('../../data/qaps.csv')

In [23]:
docs_index = docs_index.loc[lambda df: df['kind'] == 'gutenberg']
questions = questions.loc[lambda df: df['question'].str.startswith('Who')]
questions = questions.merge(docs_index.loc[:, ['document_id']], how='inner', on='document_id')

In [24]:
docs_index.head()

,document_id,set,kind,story_url,story_file_size,wiki_url,wiki_title,story_word_count,story_start,story_end
1,0029bdbe75423337b551e42bb31f9a102785376f,train,gutenberg,http://www.gutenberg.org/ebooks/21572.txt.utf-8,814507,http://en.wikipedia.org/wiki/Percival_Keene,Percival Keene,173334,Produced by Nick,new eBooks .
2,00936497f5884881f1df23f4834f6739552cee8b,train,gutenberg,http://www.gutenberg.org/ebooks/3526.txt.utf-8,566874,http://en.wikipedia.org/wiki/Five_Weeks_in_a_B...,Five Weeks in a Balloon,112898,Produced by Judy,new eBooks .
3,00950a3641e6a28b04a6fabf6334140e2deaa9fd,train,gutenberg,http://www.gutenberg.org/ebooks/42188.txt.utf-8,90192,http://en.wikipedia.org/wiki/Shadows_in_the_Mo...,Shadows in the Moonlight (story),17670,Produced by Greg,new eBooks .
6,00fb61fa7bee266ad995e52190ebb73606b60b70,valid,gutenberg,http://www.gutenberg.org/ebooks/3771.txt.utf-8,372868,http://en.wikipedia.org/wiki/Cynthia's_Revels,Cynthia's Revels,74928,Produced by Sue,new eBooks .
7,014de1a8802c05ff64efa047e9290fb7fccea2b4,test,gutenberg,http://www.gutenberg.org/ebooks/1329.txt.utf-8,560685,http://en.wikipedia.org/wiki/A_Voyage_to_Arcturus,A Voyage to Arcturus,113790,Produced by An,new eBooks .


In [25]:
questions.head()

,document_id,set,question,answer1,answer2,question_tokenized,answer1_tokenized,answer2_tokenized
0,0029bdbe75423337b551e42bb31f9a102785376f,train,Who is Miss Delmer?,the elderly spinster aunt of the Earl de Verse...,She's Captail Delmar's aunt.,Who is Miss Delmer ?,the elderly spinster aunt of the Earl de Verse...,She s Captail Delmar s aunt .
1,0029bdbe75423337b551e42bb31f9a102785376f,train,Who does Arabella Mason wed?,"Ben Keene, Delmar's valet",Ben Keene,Who does Arabella Mason wed ?,"Ben Keene , Delmar s valet",Ben Keene
2,0029bdbe75423337b551e42bb31f9a102785376f,train,Who is the bully that steals Percival's lunch?,"his teacher, Mr. O'Gallagher",The schoolmaster,Who is the bully that steals Percival s lunch ?,"his teacher , Mr. O'Gallagher",The schoolmaster
3,0029bdbe75423337b551e42bb31f9a102785376f,train,Who does Percival convince the Pirates to spare?,a rich Dutch merchant and his daughter Minnie,A Dutch Merchant and his daughter,Who does Percival convince the Pirates to spare ?,a rich Dutch merchant and his daughter Minnie,A Dutch Merchant and his daughter
4,0029bdbe75423337b551e42bb31f9a102785376f,train,Who lives at Madeline Hall?,Miss Delmar,Miss Delmar,Who lives at Madeline Hall ?,Miss Delmar,Miss Delmar


In [30]:
questions.loc[500:520, ['document_id', 'question', 'answer1', 'answer2']]

,document_id,question,answer1,answer2
500,15b1a5fe8ab222deba12f514f8a2311f901ab1d2,Who eventually helps Seaton and his friends to...,The Kondalians,Kondalians
501,15b1a5fe8ab222deba12f514f8a2311f901ab1d2,"Who is an antagonist opposing Dick Seaton, the...",Marc DuQuesne,"Marc ""Blackie"" DuQuesne"
502,15b1a5fe8ab222deba12f514f8a2311f901ab1d2,"Who kidnap Dick Seaton's fiancee, Dorothy Vane...",Marc DuQuesne,DuQuesne.
503,15b1a5fe8ab222deba12f514f8a2311f901ab1d2,Who does Martin Crane marry to?,Margaret Spencer,"Margaret ""Peg"" Spencer"
504,15b1a5fe8ab222deba12f514f8a2311f901ab1d2,Who generated the field that seaton realized h...,DuQuesne,"Marc ""Blackie"" DuQuesne"
505,15b1a5fe8ab222deba12f514f8a2311f901ab1d2,Who is seaton's millionaire friend?,Martin Crane,Martin Crane
506,16380f54b88ffa64ca4cd82d3dc5481e78ad5118,Who was Benham father?,A Schoolmaster,schoolmaster
507,16380f54b88ffa64ca4cd82d3dc5481e78ad5118,Who left part of his fourtune to Ms Benham?,Nolan,Nolan
508,16380f54b88ffa64ca4cd82d3dc5481e78ad5118,Who was Amanda?,Benham's wife,a young woman he loved passionately
509,16380f54b88ffa64ca4cd82d3dc5481e78ad5118,Who was Prothero?,Benham's friend.,Benham's schoolboy friend


In [27]:
len(docs_index)

783

In [26]:
len(questions)

5707

In [29]:
questions.set.value_counts()

train    32747
test     10557
valid     3461
Name: set, dtype: int64

In [32]:
#vocab = ['<unk>', '<s>', '</s>']
vocab = []
dataset = 'test'

for question in questions.loc[lambda df: df['set'] == dataset, 'question_tokenized']:
    for token in question.split(' '):
        vocab.append(token)
        
for answer in questions.loc[lambda df: df['set'] == dataset, 'answer1_tokenized']:
    for token in question.split(' '):
        vocab.append(token)
        
for answer in questions.loc[lambda df: df['set'] == dataset, 'answer2_tokenized']:
    for token in question.split(' '):
        vocab.append(token)

#vocab = Counter(vocab)
vocab = list(set(vocab))
vocab = ['<unk>', '<s>', '</s>'] + vocab

with open('../../data/vocab_'+dataset+'.q', 'w', encoding='utf-8') as f:
    for v in vocab:
        f.write('{}\n'.format(v))
        
with open('../../data/tokenized_'+dataset+'.q', 'w', encoding='utf-8') as f:
    for q in questions.loc[lambda df: df['set'] == dataset, 'question_tokenized']:
        f.write('{}\n'.format(q))
        
with open('../../data/tokenized_'+dataset+'.a', 'w', encoding='utf-8') as f:
    for a in questions.loc[lambda df: df['set'] == dataset, 'answer1_tokenized']:
        f.write('{}\n'.format(a))

In [5]:
filename = '../../data/glove.6B.50d.txt'

def loadGloVe(filename):
    vocab = []
    embd = []
    with codecs.open(filename, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f.readlines():
            row = line.strip().split(' ')
            vocab.append(row[0])
            embd.append(row[1:])
    return vocab,embd

vocab,embd = loadGloVe(filename)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)

In [6]:
embedding.shape

(400000, 50)

In [7]:
vocab_size

400000